In [977]:
import pandas as pd
import altair as alt

In [993]:
def df_date(df):
    df['date'] = pd.to_datetime(df['date'])

    
def data_time_choose(df,time):
    df[df['date']>time]

In [1008]:
### Read data
cpi = pd.read_csv('data/cpi.csv',index_col=0) # per month from 1913-01-01 to 2022-12-01, total 1320 observations
unemployment = pd.read_csv('data/unemployment.csv',index_col=0)# per month from 1948-01-01 to 2022-12-01, total 900 observations
gdp = pd.read_csv('data/gdp.csv',index_col=0)# per quarter from 1947-01-01 to 2022-07-01, total 303 observations
fund_rate = pd.read_csv('data/fundrate.csv',index_col=0) # per month from 1954-07-01 to 2022-12-01, total 822 observations
retail = pd.read_csv('data/retail.csv',index_col=0) # per month from 1992-01-01 to 2022-12-01, total 372 observations
durables = pd.read_csv('data/durables.csv',index_col=0) # per month from 1992-02-01 to 2022-11-01, total 370 observations
SP500 = pd.read_csv('data/SP500.csv') # per day from 2027-12-30 to 2023-01-13, total 1771 observations

In [1009]:
cpi=cpi.rename(columns={"value": "cpi"})
unemployment=unemployment.rename(columns={"value": "unemployment"})
gdp=gdp.rename(columns={"value": "gdp"})
fund_rate=fund_rate.rename(columns={"value": "fund_rate"})
retail=retail.rename(columns={"value": "retail"})
durables=durables.rename(columns={"value": "durables"})


In [1010]:
cpi=cpi[(cpi['date']>="1992-02-01") & (cpi['date']<"2022-12-01")]
unemployment=unemployment[(unemployment['date']>"1992-02-01") & (unemployment['date']<"2022-12-01")]
gdp=gdp[(gdp['date']>="1992-01-01") & (gdp['date']<"2022-12-01")]
fund_rate=fund_rate[(fund_rate['date']>="1992-02-01") & (fund_rate['date']<"2022-12-01")]
retail=retail[(cpi['date']>="1992-02-01") & (retail['date']<"2022-12-01")]
durables=durables[(cpi['date']>="1992-02-01") & (durables['date']<"2022-12-01")]

/var/folders/yw/73qbgz6d7nn4vhww_7_1sy_w0000gn/T/ipykernel_2759/2482163783.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  durables=durables[(cpi['date']>="1992-02-01") & (durables['date']<"2022-12-01")]


In [1011]:
gdp.loc[len(gdp.index)] = ['2022-10-01',60422]
gdp = gdp.sort_values(by='date')


gdp = pd.DataFrame(gdp)
gdp['date'] = pd.to_datetime(gdp['date'])
gdp = gdp.set_index('date')

# Use the resample() function to convert the data to monthly frequency
gdp= gdp.resample('M').mean()

# You can also use interpolate() function to fill the missing values
gdp= gdp.interpolate(method='polynomial', order=2)
gdp = gdp.reset_index('date')
gdp = gdp.drop(gdp.index[:2])

In [1012]:
merged_df = pd.merge(cpi, unemployment, on='date')
merged_df = pd.merge(merged_df , fund_rate, on='date')
merged_df = pd.merge(merged_df ,retail, on='date')
merged_df = pd.merge(merged_df , durables, on='date')
merged_df = merged_df.sort_values(by='date')
merged_df['gdp']=gdp["gdp"].tolist()

In [1013]:
SP500['Date'] = pd.to_datetime(SP500['Date'])
SP500.set_index('Date', inplace=True)
# Resample the data to the 'M' frequency (month end)
SP500 = SP500.resample('M').mean()
SP500 = SP500.reset_index('Date')

SP500=SP500[(SP500['Date']>="1992-03-01") & (SP500['Date']<"2022-11-01")]

merged_df['SP500']=SP500["Close"].tolist()

In [1014]:
df=merged_df
df

,date,cpi,unemployment,fund_rate,retail,durables,gdp,SP500
367,1992-03-01,139.300,7.4,3.98,142488,129309,37522.658754,407.364545
366,1992-04-01,139.500,7.4,3.73,147175,123268,37607.000000,407.407144
365,1992-05-01,139.700,7.6,3.82,152420,122274,37685.056118,414.814502
364,1992-06-01,140.200,7.8,3.76,151849,131268,37754.698968,408.274547
363,1992-07-01,140.500,7.7,3.25,152586,103025,37839.000000,415.051364
...,...,...,...,...,...,...,...,...
4,2022-06-01,296.311,3.6,1.21,609933,294470,60028.896998,3898.946684
3,2022-07-01,296.276,3.5,1.68,600746,249456,60181.000000,3911.729492
2,2022-08-01,296.171,3.7,2.33,613416,279323,60297.366565,4158.563084
1,2022-09-01,296.808,3.5,2.56,577370,289932,60375.953521,3850.520496


In [1015]:
df.corr()

,cpi,unemployment,fund_rate,retail,durables,gdp,SP500
cpi,1.000000,-0.004127,-0.701033,0.959939,0.805660,0.968270,0.870499
unemployment,-0.004127,1.000000,-0.478227,-0.118590,-0.292509,-0.136822,-0.275161
fund_rate,-0.701033,-0.478227,1.000000,-0.610382,-0.370164,-0.621187,-0.443608
retail,0.959939,-0.118590,-0.610382,1.000000,0.831966,0.947136,0.914792
durables,0.805660,-0.292509,-0.370164,0.831966,1.000000,0.842198,0.737403
gdp,0.968270,-0.136822,-0.621187,0.947136,0.842198,1.000000,0.860193
SP500,0.870499,-0.275161,-0.443608,0.914792,0.737403,0.860193,1.000000


In [1025]:
numeric_cols=df.select_dtypes('number')
numeric_cols=numeric_cols.columns.tolist()


corrlation_plot=alt.Chart(df).mark_point(opacity=0.5,size=2).encode(
   alt.X(alt.repeat('column'),type="quantitative",scale=alt.Scale(zero=False)),
   alt.Y(alt.repeat('row'),type="quantitative",scale=alt.Scale(zero=False))
).properties(height=150,width=150).repeat(row=numeric_cols,column=numeric_cols).configure_axis(titleFontSize=15,labelFontSize=5)


In [1032]:
df['cpi_rate'] = df['cpi'].pct_change()
df['unemployment_rate'] = df['unemployment'].pct_change()
df['fund_rate_rate'] = df['fund_rate'].pct_change()
df['retail_rate'] = df['retail'].pct_change()
df['durables_rate'] = df['durables'].pct_change()
df['gdp_rate'] = df['durables'].pct_change()
df['SP500_rate'] = df['SP500'].pct_change()

df_rate = df.drop(df.columns[[1,2,3,4,5,6,7]],axis=1)

In [1029]:
df_rate = df_rate.drop(df_rate.index[0])
df_rate

,date,cpi_rate,unemployment_rate,fund_rate_rate,retail_rate,durables_rate,SP500_rate,gdp_rate
366,1992-04-01,0.001436,0.000000,-0.062814,0.032894,-0.046718,0.000105,-0.046718
365,1992-05-01,0.001434,0.027027,0.024129,0.035638,-0.008064,0.018182,-0.008064
364,1992-06-01,0.003579,0.026316,-0.015707,-0.003746,0.073556,-0.015766,0.073556
363,1992-07-01,0.002140,-0.012821,-0.135638,0.004854,-0.215155,0.016599,-0.215155
362,1992-08-01,0.002847,-0.012987,0.015385,-0.000721,0.102868,0.006929,0.102868
...,...,...,...,...,...,...,...,...
4,2022-06-01,0.013736,0.000000,0.571429,-0.007390,0.115539,-0.035000,0.115539
3,2022-07-01,-0.000118,-0.027778,0.388430,-0.015062,-0.152864,0.003279,-0.152864
2,2022-08-01,-0.000354,0.057143,0.386905,0.021090,0.119729,0.063101,0.119729
1,2022-09-01,0.002151,-0.054054,0.098712,-0.058763,0.037981,-0.074074,0.037981


In [1030]:
df_rate.corr()

,cpi_rate,unemployment_rate,fund_rate_rate,retail_rate,durables_rate,SP500_rate,gdp_rate
cpi_rate,1.000000,-0.158585,0.324687,-0.130240,0.112940,0.052508,0.112940
unemployment_rate,-0.158585,1.000000,-0.325645,-0.074597,-0.157127,-0.026402,-0.157127
fund_rate_rate,0.324687,-0.325645,1.000000,0.072416,0.139273,0.002119,0.139273
retail_rate,-0.130240,-0.074597,0.072416,1.000000,0.455028,0.059755,0.455028
durables_rate,0.112940,-0.157127,0.139273,0.455028,1.000000,-0.005913,1.000000
SP500_rate,0.052508,-0.026402,0.002119,0.059755,-0.005913,1.000000,-0.005913
gdp_rate,0.112940,-0.157127,0.139273,0.455028,1.000000,-0.005913,1.000000


In [1031]:
df

,date,cpi,unemployment,fund_rate,retail,durables,gdp,SP500,cpi_rate,unemployment_rate,fund_rate_rate,retail_rate,durables_rate,SP500_rate,gdp_rate
367,1992-03-01,139.300,7.4,3.98,142488,129309,37522.658754,407.364545,NaN,NaN,NaN,NaN,NaN,NaN,NaN
366,1992-04-01,139.500,7.4,3.73,147175,123268,37607.000000,407.407144,0.001436,0.000000,-0.062814,0.032894,-0.046718,0.000105,-0.046718
365,1992-05-01,139.700,7.6,3.82,152420,122274,37685.056118,414.814502,0.001434,0.027027,0.024129,0.035638,-0.008064,0.018182,-0.008064
364,1992-06-01,140.200,7.8,3.76,151849,131268,37754.698968,408.274547,0.003579,0.026316,-0.015707,-0.003746,0.073556,-0.015766,0.073556
363,1992-07-01,140.500,7.7,3.25,152586,103025,37839.000000,415.051364,0.002140,-0.012821,-0.135638,0.004854,-0.215155,0.016599,-0.215155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2022-06-01,296.311,3.6,1.21,609933,294470,60028.896998,3898.946684,0.013736,0.000000,0.571429,-0.007390,0.115539,-0.035000,0.115539
3,2022-07-01,296.276,3.5,1.68,600746,249456,60181.000000,3911.729492,-0.000118,-0.027778,0.388430,-0.015062,-0.152864,0.003279,-0.152864
2,2022-08-01,296.171,3.7,2.33,613416,279323,60297.366565,4158.563084,-0.000354,0.057143,0.386905,0.021090,0.119729,0.063101,0.119729
1,2022-09-01,296.808,3.5,2.56,577370,289932,60375.953521,3850.520496,0.002151,-0.054054,0.098712,-0.058763,0.037981,-0.074074,0.037981


In [1033]:
df.to_csv('df.csv',index=None)